# YoloV5 to Tflite

In [1]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Load Trained model

In [3]:
from pathlib import Path

MODEL_PATH = Path("runs/train/exp")
YOLO_PATH = MODEL_PATH / "weights"
model = torch.hub.load('ultralytics/yolov5', 'custom', path=YOLO_PATH / "best.pt") 
model

Using cache found in C:\Users\Raruu/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-3-29 Python-3.10.10 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

### To ONNX

In [4]:
import onnx, onnx_tf, torch
import tensorflow as tf
from pathlib import Path

TFLITE_DIR = MODEL_PATH / "tflite"
TFLITE_DIR.mkdir(parents=True, exist_ok=True)

MODEL_PATH_ONNX = TFLITE_DIR /  f"model.onnx"
MODEL_PATH_TF = TFLITE_DIR / f"model.tf"

input_shape = torch.randn(1, 3, 640, 640).to(device)

torch.onnx.export(model, input_shape, str(MODEL_PATH_ONNX), 
                  opset_version=11, 
                  input_names=['input'], output_names=['output'],
                #   dynamic_axes={'input' : {0 : 'batch_size'},
                #                 'output' : {0 : 'batch_size'}}
                  )



C:\Users\Raruu/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:650: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  y = self.model(im, augment=augment, visualize=visualize) if augment or visualize else self.model(im)
C:\Users\Raruu/.cache\torch\hub\ultralytics_yolov5_master\models\yolo.py:100: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:


### To TfLite

In [5]:
## METHOD 2 - To TensorFlow freezGraph - https://stackoverflow.com/a/58576060
import pathlib as Path

PB_OUTPUT = TFLITE_DIR / "x.pb"
MODEL_PATH_TFLITE = TFLITE_DIR / f"RPS3_yolov5s.tflite"
!onnx-tf convert -i $MODEL_PATH_ONNX -o  $PB_OUTPUT
converter = tf.lite.TFLiteConverter.from_saved_model(str(PB_OUTPUT))
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# https://www.tensorflow.org/lite/guide/get_started#4_optimize_your_model_optional

tf_lite_model = converter.convert()
open(str(MODEL_PATH_TFLITE), 'wb').write(tf_lite_model)




d:\program files\python\python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
d:\program files\python\python310\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you

7838960

# Train YoloV5
#### Clone YoloV5 [Repo](https://github.com/ultralytics/yolov5.git)
[How To Train](https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/)

In [ ]:
!python train.py --img 640 --epochs 40 --data data.yaml --weights yolov5s.pt

In [7]:
# !python export.py --weights runs/train/exp3/weights/RPS3_yolov5s.pt --include tflite --img 640

In [25]:
!python val.py --weights runs/train/exp3/weights/RPS3_yolov5s.pt --data data.yaml

val: data=data.yaml, weights=['runs/train/exp3/weights/RPS3_yolov5s.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  v7.0-295-gac6c4383 Python-3.10.10 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs

val: Scanning D:\Sementara\Coding\Yolo\yolov5rps\valid\labels...:   0%|          | 0/2091 [00:00<?, ?it/s]
val: Scanning D:\Sementara\Coding\Yolo\yolov5rps\valid\labels... 1 images, 0 backgrounds, 0 corrupt:   0%|          | 1/2091 [00:05<3:18:42,  5.70s/it]
val: Scanning D:\Sementara\Coding\Yolo\yolov5rps\valid\labels... 36 images, 0 backgrounds, 0 corrupt:   2%|▏         | 36/2091 [00:05<03:55,  8.72it/s]
val: Scanning D:\Sementara\Coding\Y